In [7]:
# Re-run the gif creation process due to earlier interruption
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio.v2 as imageio
import io

# Re-load the CSV file
fulldatasetpath = '/Volumes/Data_Drive/datasets/VIDIMU'
file_path = os.path.join(fulldatasetpath,'dataset','videoandimusyncrop/S40/S40_A01_T01.csv')
df = pd.read_csv(file_path)

# List of updated joint names
joint_names_updated = ['pelvis', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 
                       'torso', 'neck', 'nose', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 
                       'left_wrist', 'right_wrist']

# Extract the XYZ coordinates for each joint
joint_coords_updated = {}
for joint in joint_names_updated:
    joint_coords_updated[joint] = df[[f'{joint}_x', f'{joint}_y', f'{joint}_z']].values

# Get the min and max values for each axis (for consistent plot scaling)
x_min, x_max = df[[f'{joint}_x' for joint in joint_names_updated]].min().min(), df[[f'{joint}_x' for joint in joint_names_updated]].max().max()
y_min, y_max = df[[f'{joint}_y' for joint in joint_names_updated]].min().min(), df[[f'{joint}_y' for joint in joint_names_updated]].max().max()
z_min, z_max = df[[f'{joint}_z' for joint in joint_names_updated]].min().min(), df[[f'{joint}_z' for joint in joint_names_updated]].max().max()

# Function to plot skeleton without setting a fixed view
def plot_skeleton_frame_dynamic(ax, frame_idx):
    joints = {joint: joint_coords_updated[joint][frame_idx] for joint in joint_names_updated}

    # Skeleton connections (edges between joints)
    connections_updated = [
        ('pelvis', 'left_hip'), ('pelvis', 'right_hip'),
        ('left_hip', 'left_knee'), ('right_hip', 'right_knee'),
        ('left_knee', 'left_ankle'), ('right_knee', 'right_ankle'),
        ('pelvis', 'torso'), ('torso', 'neck'), ('neck', 'nose'),
        ('neck', 'left_shoulder'), ('neck', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'), ('right_shoulder', 'right_elbow'),
        ('left_elbow', 'left_wrist'), ('right_elbow', 'right_wrist')
    ]

    xs, ys, zs = [], [], []
    for joint in joint_names_updated:
        x, y, z = joints[joint]
        xs.append(x)
        ys.append(y)
        zs.append(z)

    # Plot joints and connections
    ax.scatter(xs, ys, zs, color='blue')
    for conn in connections_updated:
        x_vals = [joints[conn[0]][0], joints[conn[1]][0]]
        y_vals = [joints[conn[0]][1], joints[conn[1]][1]]
        z_vals = [joints[conn[0]][2], joints[conn[1]][2]]
        ax.plot(x_vals, y_vals, z_vals, color='red')

    # Set the axis limits dynamically based on the entire dataset
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_zlim([z_min, z_max])

    # Hide the axis
    ax.set_axis_off()

# Create gif with dynamic movement and no fixed view
gif_images_dynamic = []
for frame in range(0, df.shape[0], 10):  # Every 10th frame for speed
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the skeleton for this frame
    plot_skeleton_frame_dynamic(ax, frame)

    # Save figure to a buffer instead of a file
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    gif_images_dynamic.append(imageio.imread(buf))
    buf.close()
    plt.close()

# Save the gif without temporary files
imageio.mimsave('skeleton_3d_dynamic.gif', gif_images_dynamic, fps=10)